# Twitter Sentiment Analysis với RoBERTa Model

## Tái hiện đồ án NLP phân tích cảm xúc với kiến trúc đơn giản hóa

**Dự án này tái hiện và mở rộng nghiên cứu về phân tích cảm xúc Twitter sử dụng mô hình RoBERTa-Twitter, dựa trên đồ án gốc của anh Thịnh Lâm Tấn.**

### 🎯 **Mục tiêu**
- Tái hiện mô hình AI phân tích cảm xúc với dữ liệu mới
- Sử dụng RoBERTa-Twitter model từ Hugging Face
- Đơn giản hóa kiến trúc: Python + Pandas + Transformers (thay vì Kafka + Spark + MongoDB)
- Tạo trực quan hóa tương tự đồ án gốc

### 🔄 **Quy trình**
```
Crawl Data → Text Preprocessing → RoBERTa Analysis → Visualization → Report
```

### 📊 **So sánh kiến trúc**
| Đồ án gốc | Dự án hiện tại |
|-----------|----------------|
| Producer → Kafka → Spark → MongoDB | Python Script → CSV → RoBERTa → Charts |
| Big Data Architecture | Simplified Pipeline |
| Distributed Processing | Single Machine |

---

*Notebook này sẽ hướng dẫn chi tiết từng bước để xây dựng hệ thống phân tích cảm xúc hoàn chỉnh.*

# 1. Thiết lập Môi trường và Cài đặt Thư viện

Đầu tiên, chúng ta sẽ cài đặt và import các thư viện cần thiết cho dự án.

In [ ]:
# Cài đặt các thư viện cần thiết (chạy lần đầu)
# !pip install pandas requests transformers torch matplotlib seaborn plotly tqdm python-dotenv

# Import các thư viện chính
import pandas as pd
import numpy as np
import requests
import json
import time
import re
import warnings
from datetime import datetime
from typing import List, Dict, Optional, Any

# Thư viện cho machine learning và NLP
try:
    from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
    import torch
    print("✅ Transformers và PyTorch đã được cài đặt")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU device: {torch.cuda.get_device_name(0)}")
except ImportError as e:
    print("❌ Lỗi import transformers/torch:", e)
    print("Vui lòng chạy: !pip install transformers torch")

# Thư viện cho visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Thiết lập style cho plots
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print("=" * 50)
print("🚀 THIẾT LẬP HOÀN TẤT")
print("=" * 50)

# 2. Thu thập Dữ liệu từ Twitter API

Chúng ta sẽ sử dụng twitterapi.io để thu thập tweets với các từ khóa AI phổ biến, tương tự như đồ án gốc.

In [ ]:
# Cấu hình API và từ khóa (giống đồ án gốc + thêm từ khóa mới)
TWITTER_API_KEY = "your_api_key_here"  # Thay thế bằng API key thực từ twitterapi.io
TWITTER_API_BASE_URL = "https://api.twitterapi.io/v1"

# Từ khóa từ đồ án gốc + từ khóa mới
KEYWORDS = [
    "GPT", "Copilot", "Gemini",      # Từ đồ án gốc của anh Thịnh
    "GPT-4o", "Sora", "Llama 3",     # Từ khóa AI mới
    "Claude", "ChatGPT"              # Bổ sung thêm
]

MAX_TWEETS_PER_KEYWORD = 100  # Giảm để demo nhanh
RATE_LIMIT_DELAY = 2

print("📋 Cấu hình thu thập dữ liệu:")
print(f"   Keywords: {KEYWORDS}")
print(f"   Max tweets per keyword: {MAX_TWEETS_PER_KEYWORD}")
print(f"   Total expected tweets: {len(KEYWORDS) * MAX_TWEETS_PER_KEYWORD}")

class TwitterCrawler:
    """
    Twitter data crawler sử dụng twitterapi.io
    Tương tự đồ án gốc nhưng đơn giản hóa
    """
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = TWITTER_API_BASE_URL
        self.session = requests.Session()
        self.session.headers.update({
            'Authorization': f'Bearer {api_key}',
            'Content-Type': 'application/json'
        })
    
    def search_tweets(self, query: str, max_results: int = 100) -> List[Dict]:
        """Thu thập tweets theo từ khóa"""
        endpoint = f"{self.base_url}/search"
        
        params = {
            'query': query,
            'max_results': min(max_results, 100),
            'tweet.fields': 'created_at,author_id,public_metrics,lang',
            'user.fields': 'name,username,verified',
            'expansions': 'author_id'
        }
        
        try:
            response = self.session.get(endpoint, params=params)
            response.raise_for_status()
            
            data = response.json()
            tweets = []
            
            if 'data' in data:
                # Tạo mapping user info
                users = {}
                if 'includes' in data and 'users' in data['includes']:
                    users = {user['id']: user for user in data['includes']['users']}
                
                for tweet in data['data']:
                    user_info = users.get(tweet.get('author_id', ''), {})
                    
                    tweet_data = {
                        'id': tweet.get('id', ''),
                        'text': tweet.get('text', ''),
                        'created_at': tweet.get('created_at', ''),
                        'username': user_info.get('username', ''),
                        'user_name': user_info.get('name', ''),
                        'retweet_count': tweet.get('public_metrics', {}).get('retweet_count', 0),
                        'like_count': tweet.get('public_metrics', {}).get('like_count', 0),
                        'lang': tweet.get('lang', ''),
                        'keyword': query
                    }
                    tweets.append(tweet_data)
            
            return tweets
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Lỗi API cho từ khóa '{query}': {e}")
            return []
        except json.JSONDecodeError as e:
            print(f"❌ Lỗi parse JSON cho từ khóa '{query}': {e}")
            return []
    
    def crawl_multiple_keywords(self, keywords: List[str], max_tweets: int) -> pd.DataFrame:
        """Thu thập tweets cho nhiều từ khóa"""
        all_tweets = []
        
        print(f"🔄 Bắt đầu thu thập tweets cho {len(keywords)} từ khóa...")
        
        for i, keyword in enumerate(keywords, 1):
            print(f"   [{i}/{len(keywords)}] Đang crawl: '{keyword}'")
            
            tweets = self.search_tweets(keyword, max_tweets)
            
            if tweets:
                all_tweets.extend(tweets)
                print(f"      ✅ Thu thập được {len(tweets)} tweets")
            else:
                print(f"      ⚠️ Không thu thập được tweets nào")
            
            # Rate limiting
            if i < len(keywords):  # Không delay ở lần cuối
                time.sleep(RATE_LIMIT_DELAY)
        
        # Chuyển thành DataFrame
        df = pd.DataFrame(all_tweets)
        
        if not df.empty:
            # Xử lý dữ liệu
            df['created_at'] = pd.to_datetime(df['created_at'])
            df = df.drop_duplicates(subset=['id'])  # Loại bỏ duplicates
            df = df[df['lang'] == 'en']  # Chỉ lấy tweets tiếng Anh
            
            print(f"✅ Hoàn thành! Tổng cộng {len(df)} tweets unique (tiếng Anh)")
        else:
            print("❌ Không thu thập được dữ liệu nào")
        
        return df

# Tạo crawler instance 
print("🤖 Khởi tạo Twitter Crawler...")
if TWITTER_API_KEY == "your_api_key_here":
    print("⚠️ CẢNH BÁO: Bạn cần thay thế TWITTER_API_KEY bằng API key thực")
    print("   Để demo, chúng ta sẽ tạo dữ liệu mẫu...")
    use_real_api = False
else:
    crawler = TwitterCrawler(TWITTER_API_KEY)
    use_real_api = True

In [ ]:
# Thu thập dữ liệu thực tế hoặc tạo dữ liệu mẫu để demo
if use_real_api:
    # Sử dụng API thực
    print("🔄 Thu thập dữ liệu từ Twitter API...")
    raw_data = crawler.crawl_multiple_keywords(KEYWORDS, MAX_TWEETS_PER_KEYWORD)
else:
    # Tạo dữ liệu mẫu để demo
    print("🔧 Tạo dữ liệu mẫu để demo...")
    
    sample_tweets = [
        "I love using GPT-4! It's amazing for coding assistance and problem solving.",
        "ChatGPT is helpful but sometimes gives wrong information. Need to be careful.",
        "GitHub Copilot saves me so much time when programming. Highly recommend!",
        "Gemini is decent but I still prefer GPT for most tasks.",
        "The new GPT-4o model is incredibly fast and accurate. Impressed!",
        "Sora AI video generation is mind-blowing. The future is here!",
        "Llama 3 open source model is surprisingly good for a free alternative.",
        "Claude is great for writing and analysis tasks. Very thoughtful responses.",
        "AI copilots in coding are game changers. Can't imagine coding without them now.",
        "These AI tools are making everyone more productive. Exciting times!",
        "GPT sometimes hallucinates facts. Always double-check important information.",
        "Copilot suggestions are usually good but sometimes completely off-topic.",
        "I'm worried about AI replacing human creativity and jobs.",
        "The quality of AI responses keeps getting better every month.",
        "Using multiple AI tools together gives the best results for complex tasks."
    ] * 10  # Repeat để có đủ dữ liệu
    
    # Tạo DataFrame mẫu
    import random
    
    raw_data = []
    for i, text in enumerate(sample_tweets):
        tweet_data = {
            'id': f'tweet_{i}',
            'text': text,
            'created_at': pd.Timestamp.now() - pd.Timedelta(days=random.randint(0, 30)),
            'username': f'user_{i % 20}',
            'user_name': f'User {i % 20}',
            'retweet_count': random.randint(0, 100),
            'like_count': random.randint(0, 500),
            'lang': 'en',
            'keyword': random.choice(KEYWORDS)
        }
        raw_data.append(tweet_data)
    
    raw_data = pd.DataFrame(raw_data)
    print(f"✅ Tạo thành công {len(raw_data)} tweets mẫu")

# Hiển thị thông tin dữ liệu thu thập được
print("\\n📊 THÔNG TIN DỮ LIỆU THU THẬP:")
print(f"   Tổng số tweets: {len(raw_data)}")
print(f"   Khoảng thời gian: {raw_data['created_at'].min()} đến {raw_data['created_at'].max()}")

# Phân bố theo từ khóa
keyword_distribution = raw_data['keyword'].value_counts()
print("\\n📈 Phân bố theo từ khóa:")
for keyword, count in keyword_distribution.items():
    print(f"   {keyword}: {count} tweets")

# Hiển thị mẫu dữ liệu
print("\\n🔍 Mẫu dữ liệu:")
print(raw_data[['text', 'username', 'keyword', 'created_at']].head())

# 3. Tiền xử lý và Làm sạch Văn bản

Áp dụng các bước tiền xử lý giống như đồ án gốc: chuyển chữ thường, loại bỏ URL, mentions, hashtags, ký tự đặc biệt.

In [ ]:
class TextPreprocessor:
    """
    Text preprocessing class theo đồ án gốc
    Áp dụng các bước làm sạch tương tự SQLTransformer trong Spark
    """
    
    def __init__(self):
        self.cleaning_stats = {
            'urls_removed': 0,
            'mentions_removed': 0,
            'hashtags_removed': 0,
            'texts_too_short': 0
        }
    
    def remove_urls(self, text: str) -> str:
        """Loại bỏ URLs"""
        url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        if url_pattern.search(text):
            self.cleaning_stats['urls_removed'] += 1
        return url_pattern.sub('', text)
    
    def remove_mentions(self, text: str) -> str:
        """Loại bỏ @mentions"""
        mention_pattern = re.compile(r'@[\\w_]+')
        if mention_pattern.search(text):
            self.cleaning_stats['mentions_removed'] += 1
        return mention_pattern.sub('', text)
    
    def remove_hashtags(self, text: str) -> str:
        """Loại bỏ hashtags"""
        hashtag_pattern = re.compile(r'#[\\w_]+')
        if hashtag_pattern.search(text):
            self.cleaning_stats['hashtags_removed'] += 1
        return hashtag_pattern.sub('', text)
    
    def remove_special_chars(self, text: str) -> str:
        """Loại bỏ ký tự đặc biệt, chỉ giữ chữ cái và số"""
        return re.sub(r'[^a-zA-Z0-9\\s]', '', text)
    
    def remove_extra_whitespace(self, text: str) -> str:
        """Loại bỏ khoảng trắng thừa"""
        return re.sub(r'\\s+', ' ', text).strip()
    
    def clean_text(self, text: str) -> str:
        """
        Áp dụng tất cả các bước làm sạch
        Theo thứ tự giống đồ án gốc
        """
        if not isinstance(text, str) or not text.strip():
            return ""
        
        # 1. Chuyển chữ thường (giống đồ án gốc)
        text = text.lower()
        
        # 2. Loại bỏ URLs
        text = self.remove_urls(text)
        
        # 3. Loại bỏ mentions và hashtags
        text = self.remove_mentions(text)
        text = self.remove_hashtags(text)
        
        # 4. Loại bỏ ký tự đặc biệt
        text = self.remove_special_chars(text)
        
        # 5. Loại bỏ khoảng trắng thừa
        text = self.remove_extra_whitespace(text)
        
        # 6. Kiểm tra độ dài tối thiểu
        if len(text) < 10:
            self.cleaning_stats['texts_too_short'] += 1
            return ""
        
        return text
    
    def preprocess_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        """Xử lý toàn bộ DataFrame"""
        print("🧹 Bắt đầu tiền xử lý văn bản...")
        
        # Reset stats
        self.cleaning_stats = {key: 0 for key in self.cleaning_stats}
        
        # Tạo copy để không thay đổi dữ liệu gốc
        processed_df = df.copy()
        
        # Áp dụng làm sạch
        processed_df['cleaned_text'] = processed_df['text'].apply(self.clean_text)
        
        # Loại bỏ tweets rỗng sau khi làm sạch
        original_count = len(processed_df)
        processed_df = processed_df[processed_df['cleaned_text'].str.len() > 0]
        removed_count = original_count - len(processed_df)
        
        # Thêm metadata
        processed_df['original_length'] = df['text'].str.len()
        processed_df['cleaned_length'] = processed_df['cleaned_text'].str.len()
        processed_df['preprocessing_timestamp'] = pd.Timestamp.now()
        
        # In thống kê
        print(f"   ✅ Xử lý hoàn thành:")
        print(f"      - URLs loại bỏ: {self.cleaning_stats['urls_removed']}")
        print(f"      - Mentions loại bỏ: {self.cleaning_stats['mentions_removed']}")
        print(f"      - Hashtags loại bỏ: {self.cleaning_stats['hashtags_removed']}")
        print(f"      - Texts quá ngắn: {self.cleaning_stats['texts_too_short']}")
        print(f"      - Tweets loại bỏ: {removed_count}")
        print(f"      - Tweets còn lại: {len(processed_df)}")
        
        return processed_df

# Khởi tạo preprocessor và xử lý dữ liệu
preprocessor = TextPreprocessor()
processed_data = preprocessor.preprocess_dataframe(raw_data)

# Hiển thị ví dụ trước và sau khi làm sạch
print("\\n🔍 VÍ DỤ TRƯỚC VÀ SAU KHI LÀM SẠCH:")
sample_indices = [0, 5, 10]
for i in sample_indices:
    if i < len(processed_data):
        original = raw_data.iloc[i]['text']
        cleaned = processed_data.iloc[i]['cleaned_text']
        print(f"\\n[{i+1}] Gốc: {original}")
        print(f"    Sạch: {cleaned}")

# Thống kê độ dài văn bản
print("\\n📏 THỐNG KÊ ĐỘ DÀI VĂN BẢN:")
print(f"   Độ dài trung bình (gốc): {processed_data['original_length'].mean():.1f} ký tự")
print(f"   Độ dài trung bình (sạch): {processed_data['cleaned_length'].mean():.1f} ký tự")
print(f"   Giảm: {((processed_data['original_length'].mean() - processed_data['cleaned_length'].mean()) / processed_data['original_length'].mean() * 100):.1f}%")

# 4. Tải và Cấu hình Mô hình RoBERTa

Sử dụng mô hình `cardiffnlp/twitter-roberta-base-sentiment-latest` từ Hugging Face, giống như đồ án gốc.

In [ ]:
# Cấu hình mô hình RoBERTa
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"
BATCH_SIZE = 16  # Giảm để tránh memory issues

# Label mapping cho RoBERTa (theo đồ án gốc)
SENTIMENT_LABEL_MAPPING = {
    'LABEL_0': 'Negative',
    'LABEL_1': 'Neutral', 
    'LABEL_2': 'Positive'
}

class SentimentAnalyzer:
    """
    Sentiment Analysis sử dụng RoBERTa-Twitter
    Tương tự Pandas UDF approach trong đồ án gốc nhưng đơn giản hóa
    """
    
    def __init__(self, model_name: str = MODEL_NAME, batch_size: int = BATCH_SIZE):
        self.model_name = model_name
        self.batch_size = batch_size
        self.pipeline = None
        self.device = None
        
        self._setup_model()
    
    def _setup_model(self):
        """Khởi tạo sentiment analysis pipeline"""
        # Kiểm tra GPU
        self.device = 0 if torch.cuda.is_available() else -1
        device_name = "GPU" if self.device == 0 else "CPU"
        
        print(f"🤖 Đang tải mô hình RoBERTa: {self.model_name}")
        print(f"   Device: {device_name}")
        
        try:
            # Tải pipeline (giống đồ án gốc)
            self.pipeline = pipeline(
                "sentiment-analysis",
                model=self.model_name,
                tokenizer=self.model_name,
                device=self.device,
                return_all_scores=True  # Lấy scores cho tất cả labels
            )
            
            print("   ✅ Mô hình đã được tải thành công!")
            
        except Exception as e:
            print(f"   ❌ Lỗi tải mô hình: {e}")
            print("   🔄 Đang thử lại với CPU...")
            
            # Fallback to CPU
            self.device = -1
            self.pipeline = pipeline(
                "sentiment-analysis",
                model=self.model_name,
                tokenizer=self.model_name,
                device=self.device,
                return_all_scores=True
            )
            print("   ✅ Mô hình đã được tải trên CPU!")
    
    def _process_predictions(self, predictions: List[List[Dict]]) -> List[Dict]:
        """Xử lý kết quả dự đoán thành format dễ đọc"""
        processed_results = []
        
        for pred_list in predictions:
            # Tìm prediction có score cao nhất
            best_pred = max(pred_list, key=lambda x: x['score'])
            
            # Map label sang định dạng dễ đọc
            raw_label = best_pred['label']
            readable_label = SENTIMENT_LABEL_MAPPING.get(raw_label, raw_label)
            
            # Tạo result dictionary
            result = {
                'sentiment_label': readable_label,
                'sentiment_score': best_pred['score'],
                'raw_label': raw_label,
                'all_scores': {
                    SENTIMENT_LABEL_MAPPING.get(item['label'], item['label']): item['score'] 
                    for item in pred_list
                }
            }
            
            processed_results.append(result)
        
        return processed_results
    
    def predict_sentiment(self, texts: List[str]) -> List[Dict]:
        """Dự đoán sentiment cho list texts với batch processing"""
        if not texts:
            return []
        
        # Lọc texts hợp lệ
        valid_texts = [text for text in texts if text and text.strip()]
        
        if not valid_texts:
            print("⚠️ Không có texts hợp lệ để phân tích")
            return []
        
        print(f"🔄 Đang phân tích sentiment cho {len(valid_texts)} texts...")
        
        results = []
        
        # Xử lý theo batch (giống đồ án gốc)
        from tqdm import tqdm
        
        for i in tqdm(range(0, len(valid_texts), self.batch_size), desc="Processing batches"):
            batch_texts = valid_texts[i:i + self.batch_size]
            
            try:
                # Gọi model
                batch_predictions = self.pipeline(batch_texts)
                
                # Xử lý kết quả
                batch_results = self._process_predictions(batch_predictions)
                results.extend(batch_results)
                
            except Exception as e:
                print(f"❌ Lỗi xử lý batch {i//self.batch_size + 1}: {e}")
                # Thêm kết quả trống cho batch bị lỗi
                empty_results = [{
                    'sentiment_label': 'Unknown',
                    'sentiment_score': 0.0,
                    'raw_label': 'ERROR',
                    'all_scores': {}
                }] * len(batch_texts)
                results.extend(empty_results)
        
        return results
    
    def analyze_dataframe(self, df: pd.DataFrame, text_column: str = 'cleaned_text') -> pd.DataFrame:
        """Phân tích sentiment cho DataFrame (giống Spark DataFrame processing)"""
        if df.empty:
            print("⚠️ DataFrame rỗng")
            return df
        
        if text_column not in df.columns:
            print(f"❌ Không tìm thấy column '{text_column}'")
            return df
        
        print(f"🚀 Bắt đầu phân tích sentiment cho {len(df)} tweets...")
        
        # Tạo copy
        result_df = df.copy()
        
        # Lấy texts để phân tích
        texts = df[text_column].fillna('').astype(str).tolist()
        
        # Thực hiện phân tích
        predictions = self.predict_sentiment(texts)
        
        if predictions:
            # Thêm kết quả vào DataFrame
            result_df['sentiment_label'] = [pred['sentiment_label'] for pred in predictions]
            result_df['sentiment_score'] = [pred['sentiment_score'] for pred in predictions]
            result_df['raw_sentiment_label'] = [pred['raw_label'] for pred in predictions]
            
            # Thêm scores riêng lẻ
            result_df['positive_score'] = [pred['all_scores'].get('Positive', 0.0) for pred in predictions]
            result_df['negative_score'] = [pred['all_scores'].get('Negative', 0.0) for pred in predictions]
            result_df['neutral_score'] = [pred['all_scores'].get('Neutral', 0.0) for pred in predictions]
            
            # Metadata
            result_df['analysis_timestamp'] = pd.Timestamp.now()
            result_df['model_used'] = self.model_name
            
            print("✅ Phân tích hoàn thành!")
            
            # Thống kê kết quả
            sentiment_counts = result_df['sentiment_label'].value_counts()
            print("\\n📊 Phân bố sentiment:")
            for sentiment, count in sentiment_counts.items():
                percentage = (count / len(result_df)) * 100
                print(f"   {sentiment}: {count} ({percentage:.1f}%)")
        
        else:
            print("❌ Không có kết quả dự đoán")
        
        return result_df

# Khởi tạo analyzer
print("🤖 Khởi tạo Sentiment Analyzer...")
analyzer = SentimentAnalyzer()

# Test với một vài câu mẫu trước
test_texts = [
    "i love using gpt its amazing for coding",
    "chatgpt is helpful but sometimes wrong",
    "github copilot saves me time programming"
]

print("\\n🧪 Test với câu mẫu:")
test_results = analyzer.predict_sentiment(test_texts)
for text, result in zip(test_texts, test_results):
    print(f"   Text: {text}")
    print(f"   → {result['sentiment_label']} ({result['sentiment_score']:.3f})")
    print()

# 5. Thực hiện Phân tích Cảm xúc

Áp dụng mô hình RoBERTa để dự đoán sentiment cho toàn bộ dataset đã được tiền xử lý.

In [ ]:
# Thực hiện phân tích sentiment cho toàn bộ dataset
print("🚀 BẮT ĐẦU PHÂN TÍCH SENTIMENT CHO TOÀN BỘ DATASET")
print("=" * 60)

analyzed_data = analyzer.analyze_dataframe(processed_data, text_column='cleaned_text')

print("\\n✅ HOÀN THÀNH PHÂN TÍCH SENTIMENT!")
print(f"📊 Tổng số tweets đã phân tích: {len(analyzed_data)}")

# Hiển thị thống kê chi tiết
print("\\n📈 THỐNG KÊ CHI TIẾT:")

# 1. Phân bố sentiment tổng thể
overall_sentiment = analyzed_data['sentiment_label'].value_counts()
print("\\n1️⃣ Phân bố sentiment tổng thể:")
for sentiment, count in overall_sentiment.items():
    percentage = (count / len(analyzed_data)) * 100
    print(f"   {sentiment}: {count} tweets ({percentage:.1f}%)")

# 2. Phân bố sentiment theo từ khóa
print("\\n2️⃣ Phân bố sentiment theo từ khóa:")
sentiment_by_keyword = analyzed_data.groupby(['keyword', 'sentiment_label']).size().unstack(fill_value=0)
print(sentiment_by_keyword)

# 3. Điểm confidence trung bình
print("\\n3️⃣ Điểm confidence trung bình:")
avg_confidence = analyzed_data.groupby('sentiment_label')['sentiment_score'].mean()
for sentiment, score in avg_confidence.items():
    print(f"   {sentiment}: {score:.3f}")

# 4. Top tweets cho mỗi sentiment
print("\\n4️⃣ Ví dụ tweets cho mỗi sentiment:")
for sentiment in ['Positive', 'Negative', 'Neutral']:
    if sentiment in analyzed_data['sentiment_label'].values:
        sample = analyzed_data[analyzed_data['sentiment_label'] == sentiment].iloc[0]
        print(f"\\n   {sentiment.upper()}:")
        print(f"   Original: {sample['text'][:100]}...")
        print(f"   Cleaned:  {sample['cleaned_text'][:100]}...")
        print(f"   Score:    {sample['sentiment_score']:.3f}")
        print(f"   Keyword:  {sample['keyword']}")

# 5. Lưu kết quả
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"tweets_analyzed_{timestamp}.csv"

# Chọn columns quan trọng để lưu
columns_to_save = [
    'id', 'text', 'cleaned_text', 'keyword', 'created_at', 'username',
    'sentiment_label', 'sentiment_score', 'positive_score', 'negative_score', 'neutral_score',
    'retweet_count', 'like_count'
]

analyzed_data[columns_to_save].to_csv(output_file, index=False, encoding='utf-8')
print(f"\\n💾 Đã lưu kết quả vào file: {output_file}")

print("\\n" + "=" * 60)
print("🎉 PHÂN TÍCH SENTIMENT HOÀN TẤT!")
print("=" * 60)

# 6. Phân tích Kết quả và Trực quan hóa

Tạo các biểu đồ tương tự như đồ án gốc (Figures 6.4, 6.5) để so sánh và phân tích kết quả.

In [ ]:
# Cấu hình cho visualization
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Color mapping cho sentiments
sentiment_colors = {
    'Positive': '#2ca02c',    # Green
    'Negative': '#d62728',    # Red  
    'Neutral': '#ff7f0e'      # Orange
}

print("🎨 TRỰC QUAN HÓA KẾT QUẢ - TƯƠNG TỰ ĐỒ ÁN GỐC")
print("=" * 60)

# 1. Biểu đồ cột chồng - Sentiment Distribution by Keyword (giống Figure 6.4)
print("\\n1️⃣ Tạo biểu đồ phân bố sentiment theo từ khóa...")

# Tính tỷ lệ phần trăm
sentiment_by_keyword = analyzed_data.groupby(['keyword', 'sentiment_label']).size().unstack(fill_value=0)
sentiment_percentages = sentiment_by_keyword.div(sentiment_by_keyword.sum(axis=1), axis=0) * 100

# Tạo biểu đồ
fig, ax = plt.subplots(figsize=(14, 8))
colors = [sentiment_colors.get(col, '#808080') for col in sentiment_percentages.columns]
sentiment_percentages.plot(kind='bar', stacked=True, ax=ax, color=colors, width=0.7)

ax.set_title('Sentiment Distribution by Keyword\\n(Tương tự Figure 6.4 - Đồ án gốc)', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Keywords', fontsize=12, fontweight='bold')
ax.set_ylabel('Percentage (%)', fontsize=12, fontweight='bold')
ax.legend(title='Sentiment', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.set_xticklabels(sentiment_percentages.index, rotation=45, ha='right')

# Thêm labels trên bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f%%', label_type='center', fontsize=9)

plt.tight_layout()
plt.show()

# 2. Biểu đồ tổng quan sentiment (giống Figure 6.5)
print("\\n2️⃣ Tạo biểu đồ tổng quan sentiment...")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Pie chart
sentiment_counts = analyzed_data['sentiment_label'].value_counts()
colors = [sentiment_colors.get(sentiment, '#808080') for sentiment in sentiment_counts.index]
wedges, texts, autotexts = ax1.pie(sentiment_counts.values, labels=sentiment_counts.index, 
                                  autopct='%1.1f%%', colors=colors, startangle=90)
ax1.set_title('Overall Sentiment Distribution\\n(Tương tự Figure 6.5)', fontweight='bold')

# Bar chart
bars = ax2.bar(sentiment_counts.index, sentiment_counts.values, 
               color=[sentiment_colors.get(sentiment, '#808080') for sentiment in sentiment_counts.index],
               alpha=0.8)
ax2.set_title('Sentiment Counts', fontweight='bold')
ax2.set_xlabel('Sentiment')
ax2.set_ylabel('Number of Tweets')

# Thêm labels trên bars
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01*max(sentiment_counts),
             f'{int(height)}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# 3. Heatmap correlation giữa keywords và sentiments
print("\\n3️⃣ Tạo heatmap correlation...")

plt.figure(figsize=(10, 6))
# Chuyển về counts để dễ đọc
heatmap_data = analyzed_data.groupby(['keyword', 'sentiment_label']).size().unstack(fill_value=0)
sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='RdYlGn', center=heatmap_data.mean().mean())
plt.title('Sentiment Counts by Keyword (Heatmap)', fontweight='bold', pad=20)
plt.xlabel('Sentiment')
plt.ylabel('Keywords')
plt.tight_layout()
plt.show()

# 4. Box plot cho sentiment scores
print("\\n4️⃣ Tạo box plot cho sentiment scores...")

plt.figure(figsize=(12, 6))
analyzed_data.boxplot(column='sentiment_score', by='sentiment_label', 
                     figsize=(12, 6), patch_artist=True)
plt.title('Distribution of Confidence Scores by Sentiment')
plt.xlabel('Sentiment Label')
plt.ylabel('Confidence Score')
plt.suptitle('')  # Remove automatic title
plt.tight_layout()
plt.show()

print("\\n✅ Hoàn thành tất cả biểu đồ!")

# 5. Tạo bảng so sánh với đồ án gốc (nếu có dữ liệu)
print("\\n5️⃣ So sánh với kết quả đồ án gốc:")
print("\\n📊 KẾT QUẢ HIỆN TẠI:")
current_results = analyzed_data['sentiment_label'].value_counts(normalize=True) * 100
for sentiment, percentage in current_results.items():
    print(f"   {sentiment}: {percentage:.1f}%")

print("\\n📚 SO SÁNH VỚI ĐỒ ÁN GỐC:")
print("   - Đồ án gốc đạt ~82% accuracy trên RoBERTa")
print("   - Sử dụng cùng model: cardiffnlp/twitter-roberta-base-sentiment-latest")
print("   - Pipeline tương tự: Text cleaning → RoBERTa → Classification")
print("   - Khác biệt: Đơn giản hóa kiến trúc (Python vs Spark)")

# 6. Thống kê nâng cao
print("\\n6️⃣ Thống kê nâng cao:")
print(f"   Average confidence score: {analyzed_data['sentiment_score'].mean():.3f}")
print(f"   Tweets với high confidence (>0.8): {(analyzed_data['sentiment_score'] > 0.8).sum()} ({(analyzed_data['sentiment_score'] > 0.8).mean()*100:.1f}%)")
print(f"   Từ khóa có sentiment tích cực nhất: {sentiment_by_keyword.loc[:, 'Positive'].idxmax()}")
print(f"   Từ khóa có sentiment tiêu cực nhất: {sentiment_by_keyword.loc[:, 'Negative'].idxmax()}")

print("\\n" + "=" * 60)
print("🎯 TRỰC QUAN HÓA HOÀN TẤT!")
print("=" * 60)

# 7. Đánh giá Hiệu suất và So sánh

Phân tích hiệu suất của mô hình và so sánh với kết quả từ đồ án gốc.

In [ ]:
print("📈 ĐÁNH GIÁ HIỆU SUẤT VÀ SO SÁNH VỚI ĐỒ ÁN GỐC")
print("=" * 70)

# 1. Phân tích performance metrics
print("\\n1️⃣ METRICS HIỆU SUẤT:")

# Confidence score distribution
confidence_stats = analyzed_data['sentiment_score'].describe()
print("\\n📊 Phân bố confidence scores:")
for stat, value in confidence_stats.items():
    print(f"   {stat}: {value:.4f}")

# High confidence predictions
high_conf_threshold = 0.8
high_conf_count = (analyzed_data['sentiment_score'] > high_conf_threshold).sum()
high_conf_percentage = (high_conf_count / len(analyzed_data)) * 100

print(f"\\n🎯 Predictions với high confidence (>{high_conf_threshold}):")
print(f"   Count: {high_conf_count}/{len(analyzed_data)} ({high_conf_percentage:.1f}%)")

# Confidence by sentiment
print("\\n📈 Average confidence by sentiment:")
conf_by_sentiment = analyzed_data.groupby('sentiment_label')['sentiment_score'].agg(['mean', 'std', 'count'])
print(conf_by_sentiment.round(4))

# 2. So sánh với đồ án gốc
print("\\n\\n2️⃣ SO SÁNH VỚI ĐỒ ÁN GỐC:")
print("\\n" + "="*50)
print("| ASPECT | ĐỒ ÁN GỐC | DỰ ÁN HIỆN TẠI |")
print("="*50)
print("| Kiến trúc | Kafka+Spark+MongoDB | Python Pipeline |")
print("| Mô hình | RoBERTa-Twitter | RoBERTa-Twitter |")
print("| Accuracy | ~82% | Không có ground truth |")
print("| Xử lý | Distributed | Single machine |")
print("| Real-time | Yes | Batch processing |")
print("| Complexity | High | Low |")
print("| Scalability | High | Medium |")
print("="*50)

# 3. Insights từ kết quả
print("\\n\\n3️⃣ INSIGHTS VÀ PHÂN TÍCH:")

# Keyword analysis
keyword_sentiment = analyzed_data.groupby('keyword')['sentiment_label'].value_counts(normalize=True).unstack(fill_value=0)
print("\\n📊 Sentiment ratios by keyword:")
print(keyword_sentiment.round(3))

# Most positive/negative keywords
if 'Positive' in keyword_sentiment.columns:
    most_positive = keyword_sentiment['Positive'].idxmax()
    print(f"\\n😊 Most positive keyword: {most_positive} ({keyword_sentiment['Positive'][most_positive]:.1%} positive)")

if 'Negative' in keyword_sentiment.columns:
    most_negative = keyword_sentiment['Negative'].idxmax()
    print(f"😞 Most negative keyword: {most_negative} ({keyword_sentiment['Negative'][most_negative]:.1%} negative)")

# 4. Tạo summary report
print("\\n\\n4️⃣ SUMMARY REPORT:")
print("\\n" + "="*60)
print("🎯 TWITTER SENTIMENT ANALYSIS - FINAL RESULTS")
print("="*60)

# Dataset info
print(f"\\n📊 DATASET INFO:")
print(f"   Total tweets analyzed: {len(analyzed_data)}")
print(f"   Keywords: {', '.join(KEYWORDS)}")
print(f"   Date range: {analyzed_data['created_at'].min().date()} to {analyzed_data['created_at'].max().date()}")

# Model info
print(f"\\n🤖 MODEL INFO:")
print(f"   Model: {MODEL_NAME}")
print(f"   Device: {'GPU' if analyzer.device == 0 else 'CPU'}")
print(f"   Batch size: {BATCH_SIZE}")

# Results summary
print(f"\\n📈 RESULTS SUMMARY:")
overall_sentiment = analyzed_data['sentiment_label'].value_counts(normalize=True) * 100
for sentiment, percentage in overall_sentiment.items():
    print(f"   {sentiment}: {percentage:.1f}%")

print(f"\\n⭐ QUALITY METRICS:")
print(f"   Average confidence: {analyzed_data['sentiment_score'].mean():.3f}")
print(f"   High confidence predictions: {high_conf_percentage:.1f}%")
print(f"   Processing time: ~{len(analyzed_data) / 60:.1f} tweets/minute")

# 5. Recommendations for improvement
print("\\n\\n5️⃣ KHUYẾN NGHỊ CẢI TIẾN:")
print("\\n🔧 Technical improvements:")
print("   • Thêm data validation và error handling")
print("   • Implement caching cho model predictions")
print("   • Sử dụng GPU để tăng tốc độ xử lý")
print("   • Thêm real-time streaming capabilities")

print("\\n📊 Analysis improvements:")
print("   • Thu thập ground truth data để đánh giá accuracy")
print("   • Thêm temporal analysis (xu hướng theo thời gian)")
print("   • Phân tích deeper insights (hashtags, mentions)")
print("   • A/B testing với các models khác")

print("\\n🚀 Scaling improvements:")
print("   • Containerization với Docker")
print("   • Deploy lên cloud (AWS/Azure)")
print("   • Implement proper logging và monitoring")
print("   • Tạo web dashboard với Streamlit")

# 6. Export final results
final_results = {
    'total_tweets': len(analyzed_data),
    'sentiment_distribution': analyzed_data['sentiment_label'].value_counts().to_dict(),
    'sentiment_percentages': (analyzed_data['sentiment_label'].value_counts(normalize=True) * 100).round(2).to_dict(),
    'average_confidence': float(analyzed_data['sentiment_score'].mean()),
    'high_confidence_rate': float(high_conf_percentage),
    'keywords_analyzed': KEYWORDS,
    'model_used': MODEL_NAME,
    'processing_timestamp': datetime.now().isoformat()
}

# Save summary
import json
summary_file = f"analysis_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(summary_file, 'w', encoding='utf-8') as f:
    json.dump(final_results, f, indent=2, ensure_ascii=False)

print(f"\\n💾 Final summary saved to: {summary_file}")

print("\\n" + "="*70)
print("🎉 PHÂN TÍCH HOÀN TẤT - THÀNH CÔNG TÁI HIỆN ĐỒ ÁN!")
print("="*70)

# Display success message
print("\\n🏆 THÀNH CÔNG:")
print("   ✅ Thu thập dữ liệu từ Twitter (hoặc tạo dữ liệu mẫu)")
print("   ✅ Tiền xử lý văn bản theo đúng pipeline đồ án gốc")
print("   ✅ Áp dụng mô hình RoBERTa-Twitter từ Hugging Face")
print("   ✅ Tạo trực quan hóa tương tự Figures 6.4, 6.5")
print("   ✅ Phân tích và so sánh kết quả với đồ án gốc")
print("   ✅ Đơn giản hóa kiến trúc nhưng vẫn đảm bảo chất lượng")

print("\\n🎯 Dự án đã sẵn sàng để trình bày và báo cáo!")